In [1]:
import numpy as np
import pandas as pd

from env.pok_env_gym_RLLib import PokemonEnv

# 1. Jouer à la main et récupérer coordonnées à la main

In [2]:
import pygame

from pygame.locals import QUIT, KEYDOWN, K_UP, K_DOWN, K_LEFT, K_RIGHT, K_a, K_b, K_q 

# Function to map keyboard inputs to actions
def get_action():
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        elif event.type == KEYDOWN:
            if event.key == K_UP:
                return 0  
            elif event.key == K_DOWN:
                return 1  
            elif event.key == K_LEFT:
                return 2  
            elif event.key == K_RIGHT:
                return 3 
            elif event.key == K_a:
                return 4  
            elif event.key == K_b:
                return 5  
            elif event.key == K_q: 
                return 7
    return 6  

objc[29111]: Class SDLApplication is implemented in both /Users/alexandreabela/Desktop/ECM/Cours/3A_ECM/PROJET3A/pokemon_env/lib/python3.11/site-packages/sdl2dll/dll/SDL2.framework/Versions/A/SDL2 (0x168069440) and /Users/alexandreabela/Desktop/ECM/Cours/3A_ECM/PROJET3A/pokemon_env/lib/python3.11/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x17f2612c8). One of the two will be used. Which one is undefined.
objc[29111]: Class SDLAppDelegate is implemented in both /Users/alexandreabela/Desktop/ECM/Cours/3A_ECM/PROJET3A/pokemon_env/lib/python3.11/site-packages/sdl2dll/dll/SDL2.framework/Versions/A/SDL2 (0x168069490) and /Users/alexandreabela/Desktop/ECM/Cours/3A_ECM/PROJET3A/pokemon_env/lib/python3.11/site-packages/pygame/.dylibs/libSDL2-2.0.0.dylib (0x17f261318). One of the two will be used. Which one is undefined.
objc[29111]: Class SDLTranslatorResponder is implemented in both /Users/alexandreabela/Desktop/ECM/Cours/3A_ECM/PROJET3A/pokemon_env/lib/python3.11/site-packages/sdl2dll/

In [3]:
from IPython.display import clear_output

def play_game():
    pygame.init()
    
    env = PokemonEnv()

    observation, info = env.reset()

    total_reward = 0
    done = False

    while not done:

        action = get_action()

        if action == 7: 
            done = True
            continue
        
        observation, reward, terminated, done, info = env.step(action)
        total_reward += reward

        m, x, y = info['coordinates']

        print(f"m: {m}, x: {x}, y: {y}")
        clear_output(wait=True)

        #print(f"Action: {action}, Reward: {reward}, Total Reward: {total_reward}")

    env.close()

    return env.seen_coords

In [4]:
seen_cords = play_game()

{'x:1 y:1 m:40': 5, 'x:0 y:1 m:40': 6, 'x:0 y:0 m:40': 2}

# 2. Plot maps une à côté des autres

In [3]:
from IPython.display import clear_output

def play_game_with_df():
    df_data = pd.DataFrame(columns=["m", "x", "y"])
    print(df_data)

    pygame.init()
    
    env = PokemonEnv()

    observation, info = env.reset()

    total_reward = 0
    done = False

    while not done:

        action = get_action()

        if action == 7: 
            done = True
            continue
        

        observation, reward, terminated, done, info = env.step(action)
        total_reward += reward

        m, x, y = info['coordinates']

        df_data.loc[len(df_data)+1]= [m, x, y]

        print(f"m: {m}, x: {x}, y:{y}")
        clear_output(wait=True)

        #print(f"Action: {action}, Reward: {reward}, Total Reward: {total_reward}")

    env.close()

    return df_data

In [4]:
df_data = play_game_with_df()

Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
m: 0, x: 1, y:1


In [5]:
df_data.to_csv("./data_test.csv")

In [4]:
df_data = pd.read_csv("./data_test.csv")

In [5]:
def make_all_coords_arrays(df):
    return np.array(df[['x', 'y', 'm']].to_numpy().astype(np.uint8))

def game_coord_to_pixel_coord(
    x, y, map_idx, base_y):
    
    global_offset = np.array([1056-16*12, 331]) #np.array([790, -29])
    map_offsets = {
        0: np.array([0,0]), # pallet town
        1: np.array([-10, 72]), # viridian
        2: np.array([-10, 180]), # pewter
        12: np.array([0, 36]), # route 1
        13: np.array([0, 144]), # route 2
        14: np.array([30, 172]), # Route 3
        15: np.array([80, 190]), #Route 4
        33: np.array([-50, 64]), # route 22
        37: np.array([-9, 2]), # red house first
        38: np.array([-9, 25-32]), # red house second
        39: np.array([9+12, 2]), # blues house
        40: np.array([25-4, -6]), # oaks lab
        41: np.array([30, 47]), # Pokémon Center (Viridian City)
        42: np.array([30, 55]), # Poké Mart (Viridian City)
        43: np.array([30, 72]), # School (Viridian City)
        44: np.array([30, 64]), # House 1 (Viridian City)
        47: np.array([21,136]), # Gate (Viridian City/Pewter City) (Route 2)
        49: np.array([21,108]), # Gate (Route 2)
        50: np.array([21,108]), # Gate (Route 2/Viridian Forest) (Route 2)
        51: np.array([-35, 137]), # viridian forest
        52: np.array([-10, 189]), # Pewter Museum (floor 1)
        53: np.array([-10, 198]), # Pewter Museum (floor 2)
        54: np.array([-21, 169]), #Pokémon Gym (Pewter City)
        55: np.array([-19, 177]), #House with disobedient Nidoran♂ (Pewter City)
        56: np.array([-30, 163]), #Poké Mart (Pewter City)
        57: np.array([-19, 177]), #House with two Trainers (Pewter City)
        58: np.array([-25, 154]), # Pokémon Center (Pewter City)
        59: np.array([83, 227]), # Mt. Moon (Route 3 entrance)
        60: np.array([123, 227]), # Mt. Moon
        61: np.array([152, 227]), # Mt. Moon
        68: np.array([65, 190]), # Pokémon Center (Route 4)
        193: None # Badges check gate (Route 22)
    }
    if map_idx in map_offsets.keys():
        offset = map_offsets[map_idx]
    else:
        offset = np.array([0,0])
        x, y = 0, 0
    coord = global_offset + 16*(offset + np.array([x,y]))
    coord[1] = base_y - coord[1]
    return coord

In [8]:
base_coords = make_all_coords_arrays(df_data)
base_coords

array([[2, 2, 2],
       [2, 2, 2],
       [2, 2, 2],
       [2, 2, 2],
       [2, 1, 2],
       [2, 2, 2],
       [2, 2, 2],
       [1, 2, 2],
       [2, 2, 2],
       [2, 2, 2],
       [2, 1, 2],
       [2, 2, 2],
       [2, 2, 2],
       [2, 3, 0],
       [2, 3, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [1, 2, 0],
       [1, 3, 0],
       [3, 3, 0],
       [3, 3, 0],
       [2, 3, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 3, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 2, 0],
       [2, 1, 0],
       [1, 1, 0],
       [2, 1, 0],
       [2, 1, 0],
       [1, 1, 0],
       [1, 1, 0],
       [1, 2, 0],
       [1, 1, 0],
       [1, 2, 0],
       [1, 3, 0],
       [1, 1, 0],
       [1, 2, 0],
       [1, 3, 0],
       [1, 2, 0],
       [1, 3, 0],
       [1, 4, 0],
       [1, 1, 0],
       [1, 2, 0],
       [1, 2, 0],
       [1, 2, 0],
       [1, 4, 0],
       [1,

In [19]:
n = base_coords.shape[0]
game_coords = np.zeros((n,2))

for i in range(n): 
    x, y, m = base_coords[i,:]
    x_pix, y_pix = game_coord_to_pixel_coord(x, y, m, 0)
    game_coords[i,:] = [x_pix, y_pix]
    

In [20]:
game_coords

array([[  736., -3243.],
       [  736., -3243.],
       [  736., -3243.],
       [  736., -3243.],
       [  736., -3227.],
       [  736., -3243.],
       [  736., -3243.],
       [  720., -3243.],
       [  736., -3243.],
       [  736., -3243.],
       [  736., -3227.],
       [  736., -3243.],
       [  736., -3243.],
       [  896.,  -379.],
       [  896.,  -379.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  880.,  -363.],
       [  880.,  -379.],
       [  912.,  -379.],
       [  912.,  -379.],
       [  896.,  -379.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -379.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -363.],
       [  896.,  -347.],
       [  880.,  -347.],
       [  896.,  -347.],
       [  896.,  -347.],
       [  880.,  -347.],
       [  880.,  -347.],


In [ ]:
def render_video(fname, all_coords, walks, bg, inter_steps=4, add_start=True):
    debug = False
    errors = []
    sprites_rendered = 0
    with media.VideoWriter(
        f'{fname}.mov', split(bg).shape[:2], codec='prores_ks', 
        encoded_format='yuva444p', input_format='rgba', fps=60
    ) as wr:
        step_count = len(all_coords)
        state = [{'dir': 0, 'map': 40} for _ in all_coords[0]]
        pbar = tqdm(range(0, step_count))
        for idx in pbar:
            step = all_coords[idx]
            if idx > 0:
                prev_step = all_coords[idx-1]
            elif add_start:
                prev_step = np.tile(np.array([5, 3, 40]), (all_coords.shape[1], 1))
            else:
                prev_step = all_coords[idx]
            if debug:
                print('-- step --')
            for fract in np.arange(0,1,1/inter_steps):
                over = np.zeros_like(bg, dtype=np.uint8)
                for run in range(len(step)):
                    cur = step[run]
                    prev = prev_step[run]
                    # cast to regular int from uint8
                    cx, cy, px, py = map(int, [cur[0], cur[1], prev[0], prev[1]])
                    dx = cx - px
                    dy = cy - py
                    total_delta = abs(dx) + abs(dy)
                    if total_delta > 1:
                        state[run]['map'] = cur[2]
                    dx = min(max(dx, -1), 1)
                    dy = -1*min(max(dy, -1), 1)
                    if debug:
                        print(f'x: {cx} y: {cy} dx: {dx} dy: {dy}')
                    # only change direction if not moving between maps
                    if cur[2] == prev[2]:
                        if dx > 0:
                            state[run]['dir'] = 3
                        elif dx < 0:
                            state[run]['dir'] = 2
                        elif dy > 0:
                            state[run]['dir'] = 1
                        elif dy < 0:
                            state[run]['dir'] = 0

                    p_coord = game_coord_to_pixel_coord(
                        cx, -cy, state[run]['map'], over.shape[0]
                    )
                    prev_p_coord = game_coord_to_pixel_coord(
                        px, -py, prev[2], over.shape[0]
                    )
                    diff = p_coord - prev_p_coord
                    interp_coord = prev_p_coord + (fract*(diff.astype(np.float32))).astype(np.int32)
                    if np.linalg.norm(diff) > 16:
                        continue
                    error = add_sprite(
                        over, walks[state[run]['dir']],
                        interp_coord
                    )
                    if error is not None:
                        errors.append(error)
                    else:
                        sprites_rendered += 1
                wr.add_image(split(over[:,:,:]))
                perc = len(errors) / (sprites_rendered + len(errors))
                pbar.set_description(f"draws: {sprites_rendered} errors: {len(errors)}, {perc:.2%}")
    return errors

In [ ]:
def render_video(fname, all_coords, bg, inter_steps=4):
    debug = False
    errors = []
    sprites_rendered = 0
    with media.VideoWriter(f'{fname}.mov', split(bg).shape[:2], codec='prores_ks', encoded_format='yuva444p', input_format='rgba', fps=60) as wr:
        step_count = len(all_coords)
        state = [{'dir': 0, 'map': 40} for _ in all_coords[0]]
        for idx in tqdm(range(1, step_count)):
            step = all_coords[idx]
            prev_step = all_coords[idx-1]
            if debug:
                print('-- step --')
            for fract in np.arange(0,1,1/inter_steps):
                over = np.zeros_like(bg, dtype=np.uint8)
                for run in range(len(step)):
                    cur = step[run]
                    prev = prev_step[run]
                    # cast to regular int from uint8
                    cx, cy, px, py = map(int, [cur[0], cur[1], prev[0], prev[1]])
                    dx = cx - px
                    dy = cy - py
                    total_delta = abs(dx) + abs(dy)
                    if total_delta > 1:
                        state[run]['map'] = cur[2]
                    dx = min(max(dx, -1), 1)
                    dy = -1*min(max(dy, -1), 1)
                    if debug:
                        print(f'x: {cx} y: {cy} dx: {dx} dy: {dy}')
                    # only change direction if not moving between maps
                    if cur[2] == prev[2]:
                        if dx > 0:
                            state[run]['dir'] = 3
                        elif dx < 0:
                            state[run]['dir'] = 2
                        elif dy > 0:
                            state[run]['dir'] = 1
                        elif dy < 0:
                            state[run]['dir'] = 0
                    
                    p_coord = game_coord_to_pixel_coord(
                        cx, -cy, state[run]['map'], over.shape[0]
                    )
                    prev_p_coord = game_coord_to_pixel_coord(
                        px, -py, prev[2], over.shape[0]
                    )
                    diff = p_coord - prev_p_coord
                    interp_coord = prev_p_coord + (fract*(diff.astype(np.float))).astype(np.int)
                    if np.linalg.norm(diff) > 16:
                        continue
                    error = add_sprite(
                        over, walks[state[run]['dir']],
                        interp_coord
                    )
                    if error is not None:
                        errors.append(error)
                    else:
                        sprites_rendered += 1
                wr.add_image(split(over[:,:,:]))
                perc = len(errors) / (sprites_rendered + len(errors))
                pbar.set_description(f"draws: {sprites_rendered} errors: {len(errors)}, {perc:.2%}")
    return errors

# 3. Afficher coordonnées endroits visités

# 4. Plot distribution des états visités 3D

# 5. Plot distribution des états visités 2D